In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import squareform
from scipy.sparse.csgraph import connected_components
from collections import Counter


Кластеры: {0: ['Apache Kafka'], 1: ['Kafka']}
Переименованные навыки: ['Apache Kafka', 'Kafka']


In [ ]:
df = 

In [ ]:
# Шаг 1: Ваши данные
skills = ["JavaScript", "JS", "Java", "Python3", "Python", "C++", "C#", "Django", "Django Framework"]
frequencies = {"JavaScript": 50, "JS": 30, "Java": 40, "Python3": 35, "Python": 70, 
               "C++": 20, "C#": 25, "Django": 15, "Django Framework": 10}

# Шаг 2: Модели
model_names = [
    "sentence-transformers/all-MiniLM-L6-v2",
    "sentence-transformers/all-mpnet-base-v2",
    "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    "sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
    "sentence-transformers/all-MiniLM-L12-v2",
    "sentence-transformers/multi-qa-mpnet-base-dot-v1",
    "sentence-transformers/multi-qa-MiniLM-L6-cos-v1",
    "sentence-transformers/paraphrase-MiniLM-L6-v2"
]

# Шаг 3: Загрузка моделей
models = [SentenceTransformer(model_name) for model_name in model_names]

# Шаг 4: Создание эмбеддингов
embeddings_by_model = [model.encode(skills, convert_to_tensor=False) for model in models]

# Шаг 5: Рассчет сходства
similarity_matrices = [
    cosine_similarity(embeddings) for embeddings in embeddings_by_model
]

# Шаг 6: Итоговая матрица сходства
similarity_threshold = 0.85
final_similarity_matrix = np.zeros_like(similarity_matrices[0])
for sim_matrix in similarity_matrices:
    final_similarity_matrix = np.maximum(final_similarity_matrix, sim_matrix)

# Бинаризация (пороговое значение)
binary_similarity_matrix = (final_similarity_matrix >= similarity_threshold).astype(int)

# Шаг 7: Кластеризация с помощью связного графа
_, labels = connected_components(csgraph=binary_similarity_matrix, directed=False)

# Шаг 8: Группировка по кластерам
clusters = {i: [] for i in range(len(set(labels)))}
for skill, label in zip(skills, labels):
    clusters[label].append(skill)

# Шаг 9: Замена на самый популярный элемент
result_mapping = {}
for cluster_id, cluster_skills in clusters.items():
    # Считаем частоту навыков в кластере
    cluster_frequencies = {skill: frequencies.get(skill, 0) for skill in cluster_skills}
    # Находим самый популярный навык
    most_popular_skill = max(cluster_frequencies, key=cluster_frequencies.get)
    # Присваиваем всем элементам кластера это название
    for skill in cluster_skills:
        result_mapping[skill] = most_popular_skill

# Шаг 10: Преобразование данных
renamed_skills = [result_mapping[skill] for skill in skills]

# Вывод
print("Кластеры:", clusters)
print("Переименованные навыки:", renamed_skills)